In [167]:
text = ""
with open("./dataset/tinyshakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [168]:
print(f"Length of dataset: {len(text)}")

Length of dataset: 1115389


In [169]:
# Create a set of all the unique characters from the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}  # This is really sexy
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]  # encoder: takes a string and return a list of integers
decode = lambda l: "".join([itos[ch] for ch in l])  # decoder: takes a list of int and return text

After this we make our tokenizer

In [170]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

In [171]:
# Split dataset
n = int(0.9 * len(data))  # first 90% will be for training
train_data = data[:n]
test_data = data[n:]

block_size = 8

In [172]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
    context = x[:i + 1]  # i+11 because we don't want 0 and range is exclusive
    target = y[i]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [173]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel
block_size = 8  # What is the maximum context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else test_data
    random_offset = torch.randint(len(train_data) - block_size, (batch_size,))
    contexts = torch.stack([data[i:i + block_size] for i in random_offset])
    targets = torch.stack([data[i + 1:i + block_size + 1] for i in random_offset])
    return contexts, targets


xb, yb = get_batch("train")
print("Inputs:")
print(xb.shape)
print(xb)

print("Outputs:")
print(yb.shape)
print(yb)

Inputs:
torch.Size([4, 8])
tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
Outputs:
torch.Size([4, 8])
tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])


In [174]:
from torch import nn
from torch.nn import functional as F

n_embd = 32
block_size = 32
device = ""


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B, T) Tensor of integers
        token_embedding = self.token_embedding_table(idx)  # (B, T, C)
        # TODO: Continue adding positional embedding and Attention
        position_embedding = self.position_embedding_table(
            torch.arange(T, ))  # if T > block_size this will throw an error
        logits = self.lm_head(token_embedding)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel()
logits, loss = model(xb, yb)
# print(logits.shape)
# print(loss)

print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=32).numpy()[0]))


zMvHkVXc
LjGBjnXNW'Uc,jHu3jvHeCr


In [175]:
# Pytorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [176]:
batch_size = 32
for steps in range(10000):
    # Sample a batch of data
    xb, yb = get_batch("train")

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4678735733032227


In [177]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500).numpy()[0]))

IndexError: index out of range in self

Mathematical Trick In Self-Attention

In [178]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # Batch, Time, Channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [179]:
# Version 1 Brute Force
# bow mean Bag Of Words (Fuck abbr) which usually means an average of values
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t + 1]  # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [180]:
# Version 2 Use weighted sum
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x

In [181]:
# Version 3 Use softmax
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = torch.masked_fill(wei, tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)  # Since softmax will clip everything, so it sums up to one it does the same as weighted sum
xbow3 = wei @ x
xbow3.shape

torch.Size([4, 8, 2])

In [262]:
# Version 4 Self-Attention
import math
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
v = value(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) * (1 / math.sqrt(k.size(-1)))  # (B, T, 16) @ (B, 16, T) = (B, T, T)
tril = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
out = wei @ v # (B, T, T) @ (B, T, 16) = (B, T, 16)

out.shape

torch.Size([4, 8, 16])

In [182]:
# This part is about exploring matrix multiplication to give us the same result as the code abov
# But in a smarter and more efficient way.
torch.manual_seed(42)
a = torch.ones((3, 3))
a = torch.tril(a)
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("A ====")
print(a)

print("B ====")
print(b)

print("C ====")
print(c)

A ====
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
B ====
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
C ====
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [241]:
torch.manual_seed(1337)
B, T, C = 1, 2, 3
x = torch.randn(B, T, C)
linLayer = nn.Linear(C, 3, bias=False)
linLayer2 = nn.Linear(C, 3, bias=False)
print(linLayer(x))
linLayer2(x)

tensor([[[-0.1269,  0.0712, -0.3388],
         [ 0.4314,  0.3180,  0.4826]]], grad_fn=<UnsafeViewBackward0>)


tensor([[[-1.2583,  0.1711, -1.4334],
         [ 0.0079,  0.4382, -0.1077]]], grad_fn=<UnsafeViewBackward0>)

In [184]:
a = torch.ones((2, 2))
b = torch.zeros((2, 2))
# print(a) # (T, C)
b[0, 0] = 0.1
b[0, 1] = 0.2
b[1, 0] = -0.2
b[1, 1] = 0.5
print(b)  # (C, T)
print(b.T)  # (C, T)
b @ b.T  # (T, T)

tensor([[ 0.1000,  0.2000],
        [-0.2000,  0.5000]])
tensor([[ 0.1000, -0.2000],
        [ 0.2000,  0.5000]])


tensor([[0.0500, 0.0800],
        [0.0800, 0.2900]])